In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score, auc, precision_recall_curve

np.random.seed(42)

### Assignment 1

Implement the calculation of binary classification quality metrics (precision, recall, and f1)

### Solution 1

In [2]:
def precision(y_true, y_pred):
    TP = (y_pred * y_true).sum()
    FP = (y_pred * (1 - y_true)).sum()
    return TP / (TP + FP)


def recall(y_true, y_pred):
    TP = (y_pred * y_true).sum()
    FN = ((1 - y_pred) * y_true).sum()
    return TP / (TP + FN)


def f1(y_true, y_pred):
    pr = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * pr * rec / (pr + rec)

### Assignment 2

Solve the binary classification problem on the breast_cancer dataset using 4 different algorithms:

     Decision Tree (1)
     Logistic Regression (2)
     KNN (3)
     SVC (4)

Find algorithms with the best ROC AUC score and PR AUC score. As an answer, write the number $ab$, where $a$ is the best algorithm in terms of ROC AUC score, and $b$ is the best algorithm in terms of PR AUC score.

Do not forget to carry out additional data preprocessing by bringing them to a single scale using the StandardScaler.

### Solution 2

Preprocessing.

In [3]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42)


sc = StandardScaler()
sc.fit(X_train)
X_train, X_test = sc.transform(X_train), sc.transform(X_test)

Model training.

In [4]:
tree = DecisionTreeClassifier().fit(X_train, y_train)
lr = LogisticRegression().fit(X_train, y_train)
knn = KNeighborsClassifier().fit(X_train, y_train)
svm = SVC(probability=True).fit(X_train, y_train)

Here we calculate the ROC AUC score and PR AUC score for each model.

In [5]:
models = [tree, lr, knn, svm]
scores, predictions, roc_score, auc_score = {}, {}, {}, {}

i = 1
for model in models:
    scores[i] = model.predict_proba(X_test)[:,1]
    predictions[i] = model.predict(X_test)
    precision, recall, thresholds = precision_recall_curve(y_test, scores[i])
    roc_score[i] = roc_auc_score(y_test, predictions[i]).round(4)
    auc_score[i] = auc(recall, precision).round(4)
    i += 1

roc_score, auc_score

({1: 0.9438, 2: 0.9828, 3: 0.9544, 4: 0.9749},
 {1: 0.9736, 2: 0.9988, 3: 0.9864, 4: 0.998})

We can see that Logistic Regression is the best one according to both ROC and PR AUC scores.

### Assignment 3

write functions to implement micro and macro averaging of metrics in multiclass classification.

You need to implement the following functions:

     err_table:        for compiling the error matrix for the multiclass case
     micro_precision:  for micro-averaging precision
     macro_precision:  for macro-averaging precision
     micro_recall:     for recall microaveraging
     macro_recall:     for recall macroaveraging
     micro_f1:         for micro-averaging the f1-measure
     macro_f1:         for macro averaging f1-measure

As part of the solution of this task, you can only use the numpy framework

### Solution 3

1. Error_matrix

    The rows of error_matrix matrix will symbolize the true classes, and the columns - the predicted ones. The number N in the matrix cell ij will be interpreted as follows:

        - This algorithm classified N objects belonging to class i as objects of class j

In [6]:
def err_table(y_true: np.array, y_pred: np.array):
    classes = np.unique(y_true)
    n = classes.size
    
    confus_matrix = np.zeros((n, n))
    
    for i in range(n):
        ind = np.where(y_true == classes[i])[0]
        y_examined = y_pred[ind]
        
        for j in range(n):
            confus_matrix[i,j] = np.where(y_examined == classes[j])[0].size
        
    return confus_matrix

Additional methods:

In [7]:
def metrices(err_table: np.array):
    n = err_table.shape[0]
    TP, FP, FN = np.zeros((n)),  np.zeros((n)),  np.zeros((n))
    
    for i in range(n):
        TP[i] = err_table[i, i]
        FP[i] = err_table[:, i].sum() - TP[i]
        FN[i] = err_table[i, :].sum() - TP[i]
        
    return TP, FP, FN


def precision(TP, FP):
    return TP / (TP + FP)

def recall(TP, FN):
    return TP / (TP + FN)

def f1(prec, rec):
    return 2*(prec * rec)/(prec + rec)

2. Micro & macro precision, recall, f1_score

In [8]:
def micro_precision(err_table: np.array):
    TP, FP,  _ = metrices(err_table)
    return precision(TP.mean(), FP.mean())

def macro_precision(err_table: np.array):
    TP, FP, _ = metrices(err_table)
    return precision(TP, FP).mean()


def micro_recall(err_table: np.array):
    TP, _, FN = metrices(err_table)
    return recall(TP.mean(), FN.mean())

def macro_recall(err_table: np.array):
    TP, _, FN = metrices(err_table)
    return recall(TP, FN).mean()


def micro_f1(err_table: np.array):
    micro_pr, micro_rec = micro_precision(err_table), micro_recall(err_table)
    return f1(micro_pr, micro_rec)
 
def macro_f1(err_table: np.array):
    TP, FP, FN = metrices(err_table)
    precis, rec = precision(TP, FP), recall(TP, FN)
    
    f1_score = f1(precis, rec)
    return f1_score.mean()

#### Example

In [9]:
y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 2, 1, 0, 2])

ErTable = err_table(y_true, y_pred)

ErTable 

array([[2., 2., 0.],
       [3., 1., 0.],
       [1., 1., 2.]])

In [10]:
micro_precision(ErTable).round(4), macro_precision(ErTable).round(4)

(0.4167, 0.5278)

In [11]:
micro_recall(ErTable).round(4), macro_recall(ErTable).round(4)

(0.4167, 0.4167)

In [12]:
macro_f1(ErTable).round(4), micro_f1(ErTable).round(4)

(0.4389, 0.4167)